# Travelling Salesman Problem

The Travelling Salesman Problem(TLP) is a classic optimization problem, studied as far back as the 1800s. It involves finding the most efficient route through a collection of cities, visiting each city exactly once.


In [20]:
from operator import attrgetter
import numpy as np
import random

In [21]:
class Individual:
    def __init__(self, chromosome_len):
        # Create an array from 0 to n
        self._chromosome = np.arange(0, chromosome_len)
        self._fitness = -1
    
    @property
    def chromosome(self):
        return self._chromosome
    
    @property
    def chromosome_len(self):
        return len(self._chromosome)
    
    def get_gene(self, offset):
        return self._chromosome[offset]
    
    def set_gene(self, offset, gene):
        self._chromosome[offset] = gene
    
    @property
    def fitness(self):
        return self._fitness

    @fitness.setter
    def fitness(self, fitness):
        self._fitness = fitness
    
    def __str__(self):
        output = ''
        for i, v in enumerate(self._chromosome):
            output += str(v)
        return output
        

In [22]:
class Population:
    def __init__(self, population_size, chromosome_len):
        self._population_fitness = -1
        self._population = [Individual] * population_size
        
        for i in range(population_size):
            self._population[i] = Individual(chromosome_len)
        
    @property
    def individuals(self):
        return self._population
    
    def get_fittest(self, offset):
        self._population = sorted(self._population, key=attrgetter('fitness'), reverse=True)
        return self_population[offset]
    
    @property
    def population_fitness(self):
        return self._population_fitness
    
    @population_fitness.setter
    def population_fitness(self, fitness):
        self._population_fitness = fitness
    
    @property
    def size(self):
        return len(self._population)
    
    def set_individual(self, offset, individual):
        self._population[offset] = individual
    
    def get_individual(self, offset):
        return self._population[offset]
    
    def shuffle(self):
        random.shuffle(self._population)

In [23]:
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, crossover_rate, elitism_count):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
    
    def init_population(self, chromosome_len):
        '''
        creates the population with a predifined size and also chromosome len
        '''
        population = Population(self.population_size, chromosome_len)
        return population

    def calculate_fitness(self, individual, cities):
        '''
        count the fitness for each of the individual gens
        '''
        route = Route(individual, cities)
        fitness = 1 / route.get_distance()
        
        individual.fitness = fitness
        return fitness

    def eval_population(self, population, cities):
        population_fitness = 0
        
        # Loop over the population evaulating individuals and summing population fitness
        for i, individual in enumerate(population.individuals):
            population_fitness += self.calculate_fitness(individual, cities)
        average_fitness = population_fitness / population.size
        population.population_fitness = average_fitness
        
    
    def is_termination_condition_met(self, population):
        pass
    
    def select_parent(self, population):
        pass
    
    def crossover_population(self, population, chromosome_len):
        pass
    
    def mutate_population(self, population, chromosome_len):
        pass


In [24]:
from math import sqrt

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_from(city):
        '''
        returns the difference of distance from a city
        '''
        deltaX = pow(city.x - self.x, 2)
        deltaY = pow(city.y - self.y, 2)
        distance = sqrt(abs(deltaX + deltaY))
        return distance

In [25]:
class Route:
    def __init__(self, individual, cities):
        # Get individual's chromosome
        chromosome = individual.chromosome
        
        # Create route
        self.route = City[len(cities)]
        for i in range(len(chromosome)):
            self.route[gene_index] = cities[chromosome[gene_index]]
    
    def get_distance(self):
        if self.distance > 0:
            return self.distance
        
        # Loop over cities in route and calculate route distance
        total_distance = 0
        for city_index in range(len(self.route)):
            total_distance = self.route[city_index].distance_from(self.route[city_index + 1])
        
        total_distance += self.route[self.route - 1].distance_from(self.routep[0])
        
        self.distance = total_distance
        return total_distance
    
        

In [29]:
class TravellingSalesmanProblem:
    def __init__(self):
        max_generations = 3000
        num_cities = 100
        
        # Create cities at random position
        x_pos = np.random.randint(100, size=num_cities)
        y_pos = np.random.randint(100, size=num_cities)
        
        cities = [City] * num_cities # np.empty((num_cities,))
        for i, (x, y) in enumerate(list(zip(x_pos, y_pos))):
            cities[i] = City(x, y)
        
        # Initial GA
        ga = GeneticAlgorithm(100, 0.001, 0.9, 2)
        population = ga.init_population(len(cities))
        
        # TODO: Evaluate populatino
        
        # Keep track of current generation
        generation = 1
        
        # Start evolution loop
        while not ga.is_termination_condition_met(generation, max_generations):
            # TODO: Print fittest individual from population
            route = Route(population.get_fittest(0), cities)
            print('best distance: {}'.format(route.get_distance()))
            
            
            # TODO: Apply crossover
            
            # TODO: Apply mutation
            
            # TODO: Evaluate population
            ga.eval_population(population, cities)
            
            # Increment the current generation
            generation += 1
        
        # TODO: Display results
        

        print('stopped after {} generations', max_generations)
        route = Route(population.get_fittest(0), cities)
        print('best distance: {}', route.get_distance())

TravellingSalesmanProblem()

NameError: name 'max_generation' is not defined